In [1]:
import gymnasium as gym
import numpy as np
import torch
import open3d as o3d
from gymnasium import spaces
import copy
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.callbacks import BaseCallback
from gymnasium.envs.registration import register
import torch.nn as nn
import matplotlib.pyplot as plt
import copy
import sys
import os
import cv2
from datetime import datetime
sys.path.append("/home/dir/RL_CoveragePlanning/viewpointPlaygroundEnv/viewpoint_env")
from viewpointWorld import CoverageEnv

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


2024-12-23 15:18:56.979302: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-23 15:18:56.980527: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-23 15:18:57.004236: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-23 15:18:57.528809: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
register(
    id="CoverageEnv-v0",
    entry_point="viewpoint_env.viewpointWorld:CoverageEnv",
)

In [3]:
env = CoverageEnv(radius=20.0, coverage_threshold=0.98, max_steps=100, action_history_length=5)

Using test_1.obj


In [14]:
def visualize_viewpoint(mesh, observation):
    # Convert legacy TriangleMesh to tensor-based TriangleMesh
    t_mesh = o3d.t.geometry.TriangleMesh.from_legacy(mesh)
    
    # Create a color array
    colors = np.full((len(t_mesh.triangles.indices), 3), [1, 1, 0])  # Yellow for unseen faces
    colors[observation == 1] = [1, 0, 0]  # Red for seen faces
    
    # Assign colors to the mesh
    t_mesh.triangle.colors = o3d.core.Tensor(colors)
    
    # Create a visualizer
    render = o3d.visualization.rendering.OffscreenRenderer(640, 480)
    render.scene.add_geometry("mesh", t_mesh)
    
    # Set up the camera view
    camera = o3d.camera.PinholeCameraParameters()
    camera.extrinsic = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, -5],  # Move the camera back by 5 units
        [0, 0, 0, 1]
    ])
    camera.intrinsic.set_intrinsics(640, 480, 500, 500, 320, 240)
    render.setup_camera(camera.intrinsic, camera.extrinsic)
    
    # Render the scene
    img = render.render_to_image()
    
    # Display the image using matplotlib
    plt.figure(figsize=(10, 8))
    plt.imshow(np.asarray(img))
    plt.title("Viewpoint Visualization")
    plt.axis('off')
    plt.show()


In [15]:
obs, _ = env.reset()  # Reset the environment
done = False
total_reward = 0
mesh = env.mesh
i = 0
while not done:
    # Select a random action
    action = env.action_space.sample()
    
    # Step the environment with the selected action
    obs, reward, done, _, _ = env.step(action)
    
    # Accumulate total reward
    total_reward += reward
    
    visualize_viewpoint(mesh, obs)
    print(f"Iteration: {i:3} | Reward; {reward:10.2f} | Total Reward: {total_reward:10.2f}")
    i = i + 1
    # Render the environment
    # env.render()

# Close the environment after testing
env.close()


AttributeError: 'open3d.cuda.pybind.geometry.TriangleMesh' object has no attribute 'triangle_colors'